In [1]:
# %cd C:\Users\duphong.hua\Documents\phong_document\test_down

In [29]:
from ultralytics import YOLOv10  # Ensure you have the appropriate import for your YOLOv10 implementation
import requests
from io import BytesIO
from PIL import Image
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv 
import pandas as pd
# Path to the trained model
TRAINED_MODEL_PATH = 'best.pt'
model = YOLOv10(TRAINED_MODEL_PATH)
CONF_THRESHOLD = 0.3
IMG_SIZE = 640
conf_list = []
detect = []
def detect_image(url):
    conf_list = []
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))\
    # convert image to np because yolo only support np.array
    img = np.array(img)
    results = model.predict(source=img, imgsz=IMG_SIZE, conf=CONF_THRESHOLD)
    for result in results:
        len_box = len(result.boxes)       
        # detect.append(len_box)
    for detection in result.boxes:
        x1, y1, x2, y2 = map(int, detection.xyxy[0])  # Bounding box coordinates
        label = detection.cls[0]  # Class label
        conf = detection.conf[0]  # Confidence score 
        conf_list.append(conf)
        # print(f'Label is {label} with confidence score {conf:.2f}')
        return conf

def display_bounding_box(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    img = np.array(img)
    results = model.predict(source=img, imgsz=IMG_SIZE, conf=CONF_THRESHOLD)
    for result in results:
        len_box = len(result.boxes)       
        # detect.append(len_box)
    for detection in result.boxes:
        x1, y1, x2, y2 = map(int, detection.xyxy[0])  # Bounding box coordinates
        label = detection.cls[0]  # Class label
        conf = detection.conf[0]  # Confidence score
        # Draw bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        # Draw label and confidence score
        cv2.putText(img, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Display the image inline
    plt.imshow(img_rgb)
    plt.axis('off')  # Hide axes
    plt.show()        

In [ ]:
a = detect_image('https://down-tx-vn.img.susercontent.com/'+'vn-11134525-7r98o-lxfjwchrpty16f')
a.item()

0.7649893164634705

In [ ]:
# Convert BGR image to RGB
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# Display the image inline
plt.imshow(img_rgb)
plt.axis('off')  # Hide axes
plt.show()

In [33]:
df = pd.read_csv('raw_9jul.csv')
# Confidence threshold and image size
CONF_THRESHOLD = 0.1
IMG_SIZE = 640
conf_list = []
detect = []
def detect_img(img_name,name):
    # Load the image from the given path or URL
    image = cv2.imread(img_name)
    order_code = name
    # Predict using the YOLOv10 model
    results = model.predict(source=image, imgsz=IMG_SIZE, conf=CONF_THRESHOLD)
    
    # Iterate through the results and draw bounding boxes and labels
    for result in results:
        len_box = len(result.boxes)
        if len_box > 0 :
            df.loc[df['order_code'] == order_code,'is_detection'] = 1
        else: 
            df.loc[df['order_code'] == order_code,'is_detection'] = 0            
        # detect.append(len_box)
        for detection in result.boxes:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])  # Bounding box coordinates
            label = detection.cls[0]  # Class label
            conf = detection.conf[0]  # Confidence score
            conf_list.append(conf) 
            df.loc[df['order_code'] == order_code,'conf_score'] = str(conf)
            # Draw bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            
            # Draw label and confidence score
            cv2.putText(image, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    
    # Save the image with detections
    save_path = './result/'
    output_image_path = save_path + f'{order_code}.jpg'
    cv2.imwrite(output_image_path, image)
    df.to_csv(save_path+'result.csv',index=False,encoding='utf-8-sig')
    print(f'Saved detected image to {output_image_path}')
# Example usage:
# detect_img('path_to_your_image.jpg')


In [17]:
# import os 
# for i in range(0,len(df)):
    

In [10]:
# import requests 
# url = 'https://down-tx-vn.img.susercontent.com/'
# name = df['order_code'][0]
# img = url + df['image_urls_at_drop'][0]
# # detect_img(img,name)
# requests.get(img).content

In [24]:
import os
path = './actual_data/'
img = os.listdir(path)[0:100]

for i in img:
    # print(i.split('.')[0])
    detect_img(img_name=path + i,name=i.split('.')[0])
# cv2.imread(img)
# model.predict(source=path+img[0], imgsz=IMG_SIZE, conf=CONF_THRESHOLD)


0: 640x480 (no detections), 191.5ms
Speed: 3.0ms preprocess, 191.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)
Saved detected image to ./result/240709E0E1U6.jpg

0: 640x480 1 pod, 206.8ms
Speed: 3.0ms preprocess, 206.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)
Saved detected image to ./result/240709E0GSJU.jpg

0: 640x480 (no detections), 180.2ms
Speed: 4.0ms preprocess, 180.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
Saved detected image to ./result/240709E0KPDV.jpg

0: 640x480 1 pod, 179.5ms
Speed: 3.0ms preprocess, 179.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
Saved detected image to ./result/240709E1GZFK.jpg

0: 640x480 2 pods, 203.9ms
Speed: 5.0ms preprocess, 203.9ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 480)
Saved detected image to ./result/240709E33NZO.jpg

0: 640x480 (no detections), 261.1ms
Speed: 3.0ms preprocess, 261.1ms inference, 2.0ms postprocess per i

In [97]:
a = pd.read_csv('./result/'+'result.csv')
b = a[a['is_detection'].isna() == False]
b['original_image_link'] = 'https://down-tx-vn.img.susercontent.com/' + a['image_urls_at_drop']
b.to_csv('./result/'+'result.csv',encoding='utf-8-sig')
a

C:\Users\duphong.hua\AppData\Local\Temp\ipykernel_16344\2017901466.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['original_image_link'] = 'https://down-tx-vn.img.susercontent.com/' + a['image_urls_at_drop']


,submit_date,submit_ts,submit_hour,order_code,source,shipper_id,shipper_name,shipper_city_name,order_city_name,order_status,image_urls_at_drop,sender_name,cod_value,item_value,image_urls_cnt,is_detection,conf_score
0,2024-07-06,2024-07-06 23:56:31.000,23,240706EIWE43,order_shopee,41343689,Hoàng Thanh Thái Dương 0356,Ha Noi City,Ha Noi City,Delivered,vn-11134525-7r98o-lxe8g4mpgrtnc2,"""MAY-clothings""",0.0,204000.0,1,1.0,tensor(0.5919)
1,2024-07-06,2024-07-06 23:53:33.000,23,240706EGIWHT,order_shopee,19465742,Lê Hoàng Anh Tú 7262,HCM City,HCM City,Delivered,vn-11134525-7r98o-lxdzvhh1eztn65,"""Hy.Nail Korea""",0.0,198000.0,1,1.0,tensor(0.9559)
2,2024-07-06,2024-07-06 23:30:52.000,23,240706EY0ZW6,order_shopee,16593548,Võ Tuấn Bình 310,HCM City,HCM City,Delivered,vn-11134525-7r98o-lxeahx3trftn63,"""Đan Chi NailBox""",0.0,698100.0,1,1.0,tensor(0.3016)
3,2024-07-06,2024-07-06 23:18:41.000,23,240706EVJLFE,order_shopee,40366772,Đặng Văn Trường 7940,Ha Noi City,Ha Noi City,Delivered,vn-11134525-7r98o-lxe1hor4ycbdf1,"""JOLLY STORE""",0.0,90000.0,1,0.0,NaN
4,2024-07-06,2024-07-06 23:18:29.000,23,240706EARXOA,order_shopee,50750345,Trần Đức Tiến,HCM City,HCM City,Delivered,vn-11134525-7r98o-lxe49jj2h23dbd,"""Fairy.S""",71100.0,24000.0,1,1.0,tensor(0.8759)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2024-07-06,2024-07-06 19:28:54.000,19,240706EB5UXL,order_shopee,50567967,Tạ Xuân Quyển 9987,Ha Noi City,Ha Noi City,Delivered,vn-11134525-7r98o-lxe1ig8kp0ih8e,"""GMART- THẾ GIỚI ĐỒ ĂN""",108600.0,75000.0,1,NaN,NaN
996,2024-07-06,2024-07-06 19:28:50.000,19,240706EYGDD0,order_shopee,50510015,Lữ Đăng Khoa 6115,HCM City,HCM City,Delivered,vn-11134525-7r98o-lxdz5j4aggkb13,"""qossh6r6aj""",0.0,179000.0,1,NaN,NaN
997,2024-07-06,2024-07-06 19:28:45.000,19,240706EPNG12,order_shopee,40007550,Nguyễn Thái Hiền 6095,HCM City,HCM City,Delivered,vn-11134525-7r98o-lxe2pgdh3umxbb,"""CÁ MẬP PETSHOP""",189900.0,163400.0,1,NaN,NaN
998,2024-07-06,2024-07-06 19:28:45.000,19,240706ELOJHT,order_shopee,40336225,Đào Đức Nam 6918,Ha Noi City,Ha Noi City,Delivered,vn-11134525-7r98o-lxe2jfw1wkpnfa,"""Handmadebysan""",0.0,267500.0,1,NaN,NaN


In [40]:
# a = r'C:\Users\duphong.hua\Documents\phong_document\git\resp_test\test_yolo\result\240706E1FKRG.jpg'
# b = model.predict(a,imgsz=IMG_SIZE, conf=CONF_THRESHOLD) 	
# # for result in b:
#         # for detection in result.boxes:
#                 # print(detection.conf)
# len(b[0].boxes)

In [ ]:
# for result in results:
#     for detection in result.boxes:
#         x1, y1, x2, y2 = map(int, detection.xyxy[0])  # Lấy tọa độ bounding box
#         label = detection.cls[0]  # Lấy nhãn
#         conf = detection.conf[0]  # Lấy độ tin cậy


In [2]:
from function import *
import pandas as pd
detector = DetectObject('best.pt')

In [2]:
# confidence_score = detector.detect_image('https://down-tx-vn.img.susercontent.com/'+'vn-11134525-7r98o-lxfjwchrpty16f', 0.1).item()
# print(confidence_score.item())
# detector.display_image('https://down-tx-vn.img.susercontent.com/'+'vn-11134525-7r98o-lxfjwchrpty16f', 0.1)

In [18]:
web_path = 'https://down-tx-vn.img.susercontent.com/'
df = pd.read_csv('raw_test.csv')
df_length = len(df)
step = 0
for i in df['order_code']:
    image_name = df[df['order_code'] == i]['image_urls_at_drop'].to_list()
    image_url = web_path + image_name[0]
    # print(i,image_url)
    confidence_score = detector.detect_image(web_path+image_name[0],0.3)
    if confidence_score is None:
        df.loc[df['order_code'] == i,'confidence_score'] = 'No detections'
        # print(confidence_score)
    else:
        df.loc[df['order_code'] == i,'confidence_score'] = confidence_score.item()
    #     print('check')
print(f'Process done with {step} rows data')
# Create a numpy array of image URLs
# image_urls = web_path + df['image_urls_at_drop'].values

# # Detect confidence scores for all images
# confidence_scores = np.array([detector.detect_image(url, 0.3) for url in image_urls])

# # Replace None values with 'No detections' and convert tensors to their item values
# confidence_scores = np.where(confidence_scores == None, 'No detections', [score.item() if score is not None else score for score in confidence_scores])

# # Assign the confidence scores back to the dataframe
# df['confidence_score'] = confidence_scores

# print(f'Process done with {len(df)} rows data')




0: 640x480 (no detections), 173.5ms
Speed: 4.0ms preprocess, 173.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 pod, 153.7ms
Speed: 3.0ms preprocess, 153.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 145.6ms
Speed: 3.0ms preprocess, 145.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 151.1ms
Speed: 3.0ms preprocess, 151.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 172.5ms
Speed: 2.0ms preprocess, 172.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 338.3ms
Speed: 2.0ms preprocess, 338.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 278.8ms
Speed: 3.0ms preprocess, 278.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 228.4ms
Speed: 2.0ms preprocess, 228

In [25]:
df.to_csv('test_result.csv')

In [17]:
test = web_path + df.loc[df['order_code'] == '240710EXP1DZ','image_urls_at_drop'][0]
confidence_score = detector.detect_image(test, 0.3)
if confidence_score is None:
    print("No detections")
else:
    print(f"Confidence score: {confidence_score}")
# list(df[df['order_code'] == '240710EXP1DZ']['image_urls_at_drop'])
# df[df['order_code'] == i]['image_urls_at_drop']


0: 640x480 (no detections), 177.5ms
Speed: 3.0ms preprocess, 177.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
No detections
